In [1]:
# self-generated similar design, PEERS semantic matrix
import pandas as pd
import numpy as np
import pickle

In [2]:
# 1638 words
with open("../Data/wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [3]:
n = 48
v = n-8  # last 8 not tested
wordpool = np.arange(1,1639)

nsubj = 100
g1_per = 5 # 3
g2_per = 3 # 5
g3_per = 4 # 4

### Group1: Association Recog - CR

In [4]:
df_g1_study = pd.DataFrame()
df_g1_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for subj in range(nsubj):

    all_words = rng.choice(wordpool, g1_per*2*n, replace = False)

    for lst in range(g1_per):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()

        # study
        pres_words = all_words[lst*2*n:(lst+1)*2*n]
        pres_words = np.reshape(pres_words, (n,2))
        pair_order = rng.permutation([0]*int(v/2) + [1]*int(v/2))  # 0: A-B, 1: B-A (during CR)
        tmp_df_study['study_itemno1'] = pres_words[:,0]
        tmp_df_study['study_itemno2'] = pres_words[:,1]
        tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
        tmp_df_study['study_item2'] = items[pres_words[:,1]-1]    
        tmp_df_study['order'] = np.concatenate((pair_order, [-1]*8))
        tmp_df_study['pair_idx'] = np.arange(n) + lst*n
        tmp_df_study['list'] = lst
        tmp_df_study['session'] = subj
        tmp_df_study['subject'] = subj
        df_g1_study = pd.concat([df_g1_study,tmp_df_study])
        
        valid_words = pres_words[0:v,:]

        # test1: association recognition
        intact_idx = rng.permutation([True]*int(v/2) + [False]*int(v/2))
        intact_pair_idx = np.arange(v)[intact_idx] + lst*n
        intact_pairs_raw = valid_words[intact_idx,:]
        intact_pairs = valid_words[intact_idx,:]
        intact_order = pair_order[intact_idx] # 0:A-B, 1:B-A
        for i in range(intact_pairs.shape[0]):
            if intact_order[i] == 1:
                intact_pairs[i] = np.flip(intact_pairs[i])

        rearrange_idx = ~intact_idx
        rearrange_pair_idx = np.array([-1]*int(v/2))
        rearrange_pairs = valid_words[rearrange_idx,:]
        rearrange_pairs[:,1] = rng.permutation(rearrange_pairs[:,1])
        rearrange_order = pair_order[rearrange_idx] # 0:A-B, 1:B-A
        for i in range(rearrange_pairs.shape[0]):
            if rearrange_order[i] == 1:
                rearrange_pairs[i] = np.flip(rearrange_pairs[i])

        test1_pairs = np.concatenate((intact_pairs,rearrange_pairs),axis=0)
        test1_pair_idx = np.concatenate((intact_pair_idx,rearrange_pair_idx),axis=0)
        test1_type = np.array([1]*int(v/2)+[0]*int(v/2))  #1:intact, 0:rearranged
        test1_order = np.concatenate((intact_order,rearrange_order),axis=0)
        tmp_df_test1['test_itemno1'] = test1_pairs[:,0]
        tmp_df_test1['test_itemno2'] = test1_pairs[:,1]
        tmp_df_test1['test_item1'] = items[test1_pairs[:,0]-1]
        tmp_df_test1['test_item2'] = items[test1_pairs[:,1]-1]
        tmp_df_test1['correct_ans'] = test1_type
        tmp_df_test1['order'] = test1_order
        tmp_df_test1['pair_idx'] = test1_pair_idx
        tmp_df_test1['test'] = 1
        tmp_df_test1['list'] = lst
        tmp_df_test1['session'] = subj
        tmp_df_test1['subject'] = subj
        tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)
        
        # test2: cued recall
        cued_order = pair_order[intact_idx] # 0:A-B, 1:B-A
        test2_probe = np.array([intact_pairs_raw[i,cued_order[i]] for i in range(int(v/2))])
        test2_ans = np.array([intact_pairs_raw[i,1-cued_order[i]] for i in range(int(v/2))])
        tmp_df_test2['test_itemno1'] = test2_probe
        tmp_df_test2['test_itemno2'] = -1
        tmp_df_test2['test_item1'] = items[test2_probe-1]
        tmp_df_test2['test_item2'] = -1
        tmp_df_test2['correct_ans'] = test2_ans
        tmp_df_test2['order'] = cued_order
        tmp_df_test2['pair_idx'] = intact_pair_idx
        tmp_df_test2['test'] = 2
        tmp_df_test2['list'] = lst
        tmp_df_test2['session'] = subj
        tmp_df_test2['subject'] = subj
        tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)
        df_g1_test = pd.concat([df_g1_test,tmp_df_test1,tmp_df_test2])

df_g1_study.reset_index(inplace = True, drop=True)
df_g1_test.reset_index(inplace = True, drop=True)
df_g1_study['group'] = 1
df_g1_test['group'] = 1

In [5]:
# len(np.unique(df_g1_study.query("session==0")[["study_itemno1","study_itemno2"]].values))
df_g1_study.query("session==1")

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
240,354,1026,COMMANDER,PATRIOT,1,0,0,1,1,1
241,947,934,NEUTRON,MUSHROOM,1,1,0,1,1,1
242,1072,546,PIN,EYELASH,0,2,0,1,1,1
243,1376,735,STEM,INCENSE,0,3,0,1,1,1
244,867,1334,MARBLE,SNEAKER,1,4,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
475,536,818,ENGINEER,LIEUTENANT,-1,235,4,1,1,1
476,77,373,BANANA,COOKBOOK,-1,236,4,1,1,1
477,845,329,LOUNGE,CLOVE,-1,237,4,1,1,1
478,783,273,KNAPSACK,CHAIN,-1,238,4,1,1,1


In [6]:
df_g1_test.query("session==0 and list==4")

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,list,session,subject,group
240,538,1276,ERASER,SHEARS,1,0,222,1,4,0,0,1
241,358,1421,COMPASS,SWING,1,1,220,1,4,0,0,1
242,1638,883,ZUCCHINI,MEAT,0,1,-1,1,4,0,0,1
243,32,441,APPENDAGE,DASHBOARD,1,0,229,1,4,0,0,1
244,970,799,OCEAN,LANDSCAPE,1,0,201,1,4,0,0,1
245,826,1314,LINT,SKELETON,1,1,204,1,4,0,0,1
246,1110,1489,PORT,TOWEL,1,1,213,1,4,0,0,1
247,1172,1527,REBEL,TWIG,1,1,198,1,4,0,0,1
248,570,1387,FIREPLACE,STORE,0,0,-1,1,4,0,0,1
249,988,514,OUTFIT,EARRING,0,0,-1,1,4,0,0,1


### Group2: Pair Recog - CR

In [7]:
df_g2_study = pd.DataFrame()
df_g2_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for subj in range(nsubj):

    all_words = rng.choice(wordpool, g2_per*2*(n+v), replace=False)

    for lst in range(g2_per):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()

        # study
        pres_words = all_words[lst*2*(n+v):lst*2*(n+v)+2*n]
        pres_words = np.reshape(pres_words, (n,2))
        pair_order = rng.permutation([0]*int(v/2) + [1]*int(v/2))  # 0: A-B, 1: B-A (during CR)
        tmp_df_study['study_itemno1'] = pres_words[:,0]
        tmp_df_study['study_itemno2'] = pres_words[:,1]
        tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
        tmp_df_study['study_item2'] = items[pres_words[:,1]-1]
        tmp_df_study['order'] = np.concatenate((pair_order, [-1]*8))
        tmp_df_study['pair_idx'] = np.arange(n)  + lst*n
        tmp_df_study['list'] = lst
        tmp_df_study['session'] = subj
        tmp_df_study['subject'] = subj
        df_g2_study = pd.concat([df_g2_study,tmp_df_study])

        valid_words = pres_words[0:v,:]

        # test1: pair recognition
        old_pairs = valid_words.copy()
        old_order = pair_order # 0:A-B, 1:B-A
        for i in range(old_pairs.shape[0]):
            if old_order[i] == 1:
                old_pairs[i] = np.flip(old_pairs[i])
        old_pair_idx = np.arange(v) + lst*n

        new_words = all_words[lst*2*(n+v)+2*n:(lst+1)*2*(n+v)]
        new_pairs = np.reshape(new_words, (v,2))
        new_order = [-1] * v
        new_pair_idx = np.array([-1] * v)

        test1_pairs = np.concatenate((old_pairs,new_pairs),axis=0)
        test1_type = np.array([1]*v+[0]*v)  #1:old, 0:new
        test1_order = np.concatenate((old_order,new_order),axis=0)
        test1_pair_idx = np.concatenate((old_pair_idx,new_pair_idx),axis=0)
        tmp_df_test1['test_itemno1'] = test1_pairs[:,0]
        tmp_df_test1['test_itemno2'] = test1_pairs[:,1]
        tmp_df_test1['test_item1'] = items[test1_pairs[:,0]-1]
        tmp_df_test1['test_item2'] = items[test1_pairs[:,1]-1]
        tmp_df_test1['correct_ans'] = test1_type
        tmp_df_test1['order'] = test1_order
        tmp_df_test1['pair_idx'] = test1_pair_idx
        tmp_df_test1['test'] = 1
        tmp_df_test1['list'] = lst
        tmp_df_test1['session'] = subj
        tmp_df_test1['subject'] = subj
        tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

        # test2: cued recall
        cued_order = pair_order # 0:A-B, 1:B-A
        test2_probe = np.array([valid_words[i,cued_order[i]] for i in range(v)])
        test2_ans = np.array([valid_words[i,1-cued_order[i]] for i in range(v)])
        tmp_df_test2['test_itemno1'] = test2_probe
        tmp_df_test2['test_itemno2'] = -1
        tmp_df_test2['test_item1'] = items[test2_probe-1]
        tmp_df_test2['test_item2'] = -1
        tmp_df_test2['correct_ans'] = test2_ans
        tmp_df_test2['order'] = cued_order
        tmp_df_test2['pair_idx'] = np.arange(v) + lst*n
        tmp_df_test2['test'] = 2
        tmp_df_test2['list'] = lst
        tmp_df_test2['session'] = subj
        tmp_df_test2['subject'] = subj
        tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)
        df_g2_test = pd.concat([df_g2_test,tmp_df_test1,tmp_df_test2])

df_g2_study.reset_index(inplace = True, drop=True)
df_g2_test.reset_index(inplace = True, drop=True)
df_g2_study['group'] = 2
df_g2_test['group'] = 2

In [8]:
df_g2_study

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
0,243,1454,CARD,THESAURUS,0,0,0,0,0,2
1,1169,737,RAT,INFANT,0,1,0,0,0,2
2,660,1109,GREASE,PORPOISE,1,2,0,0,0,2
3,354,1499,COMMANDER,TREAD,1,3,0,0,0,2
4,1492,124,TOY,BIBLE,0,4,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...
19195,836,1195,LOBBY,ROACH,-1,187,3,99,99,2
19196,932,261,MUMMY,CATTLE,-1,188,3,99,99,2
19197,1054,1600,PHILOSOPHER,WELL,-1,189,3,99,99,2
19198,36,1021,ARCHITECT,PASTA,-1,190,3,99,99,2


In [9]:
df_g2_test.query("session==0 and list==2")

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,list,session,subject,group
240,1072,1079,PIN,PIZZA,0,-1,-1,1,2,0,0,2
241,1155,755,RACKET,JELLY,0,-1,-1,1,2,0,0,2
242,1491,290,TOWN,CHEF,0,-1,-1,1,2,0,0,2
243,305,702,CHISEL,HELMET,1,0,103,1,2,0,0,2
244,1062,778,PIE,KITCHEN,0,-1,-1,1,2,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
355,863,-1,MANAGER,-1,1367,0,117,2,2,0,0,2
356,1602,-1,WHEAT,-1,1350,0,108,2,2,0,0,2
357,729,-1,HUT,-1,174,1,104,2,2,0,0,2
358,1377,-1,STEP,-1,1068,1,130,2,2,0,0,2


### Group3: Item Recog - CR

In [10]:
df_g3_study = pd.DataFrame()
df_g3_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for subj in range(nsubj):

    all_words = rng.choice(wordpool, g3_per*(2*n+v), replace=False)

    for lst in range(g3_per):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()

        # study
        pres_words = all_words[lst*(2*n+v):lst*(2*n+v)+2*n]
        pres_words = np.reshape(pres_words, (n,2))
        pair_order = rng.permutation([0]*int(v/2) + [1]*int(v/2)) # 0: A-B, 1: B-A (during CR)
        tmp_df_study['study_itemno1'] = pres_words[:,0]
        tmp_df_study['study_itemno2'] = pres_words[:,1]
        tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
        tmp_df_study['study_item2'] = items[pres_words[:,1]-1]
        tmp_df_study['order'] = np.concatenate((pair_order, [-1]*8))
        tmp_df_study['pair_idx'] = np.arange(n) + lst*n
        tmp_df_study['list'] = lst
        tmp_df_study['session'] = subj
        tmp_df_study['subject'] = subj
        df_g3_study = pd.concat([df_g3_study,tmp_df_study])

        valid_words = pres_words[0:v,:]

        # test1: item recognition
        old_order = pair_order # 0:B, 1:A
        old_probe = np.array([valid_words[i,1-old_order[i]] for i in range(v)])
        old_pair_idx = np.arange(v) + lst*n
        new_probe = all_words[lst*(2*n+v)+2*n:(lst+1)*(2*n+v)]
        new_order = [-1] * v
        new_pair_idx = np.array([-1] * v)

        test1_probe = np.concatenate((old_probe,new_probe),axis=0)
        test1_type = np.array([1]*v+[0]*v)  #1:old, 0:new
        test1_order = np.concatenate((old_order,new_order),axis=0)
        test1_pair_idx = np.concatenate((old_pair_idx,new_pair_idx),axis=0)
        tmp_df_test1['test_itemno1'] = test1_probe
        tmp_df_test1['test_itemno2'] = -1
        tmp_df_test1['test_item1'] = items[test1_probe-1]
        tmp_df_test1['correct_ans'] = test1_type
        tmp_df_test1['order'] = test1_order
        tmp_df_test1['pair_idx'] = test1_pair_idx
        tmp_df_test1['test'] = 1
        tmp_df_test1['list'] = lst
        tmp_df_test1['session'] = subj
        tmp_df_test1['subject'] = subj
        tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

        # test2: cued recall
        cued_order = pair_order # 0:A-B, 1:B-A
        test2_probe = np.array([valid_words[i,cued_order[i]] for i in range(v)])
        test2_ans = np.array([valid_words[i,1-cued_order[i]] for i in range(v)])
        tmp_df_test2['test_itemno1'] = test2_probe
        tmp_df_test2['test_itemno2'] = -1
        tmp_df_test2['test_item1'] = items[test2_probe-1]
        tmp_df_test2['correct_ans'] = test2_ans
        tmp_df_test2['order'] = cued_order
        tmp_df_test2['pair_idx'] = np.arange(v) + lst*n
        tmp_df_test2['test'] = 2
        tmp_df_test2['list'] = lst
        tmp_df_test2['session'] = subj
        tmp_df_test2['subject'] = subj
        tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)
        df_g3_test = pd.concat([df_g3_test,tmp_df_test1,tmp_df_test2])
        
df_g3_study.reset_index(inplace = True, drop=True)
df_g3_test.reset_index(inplace = True, drop=True)
df_g3_study['group'] = 3
df_g3_test['group'] = 3

In [11]:
df_g3_study

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
0,383,1453,COSTUME,THERMOMETER,0,0,0,0,0,3
1,1122,681,PRIEST,HAMMOCK,1,1,0,0,0,3
2,1048,1610,PERCH,WILDERNESS,1,2,0,0,0,3
3,626,542,GAUZE,EWE,0,3,0,0,0,3
4,918,1130,MOSS,PROFESSIONAL,1,4,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...
14395,1111,294,PORTRAIT,CHESS,-1,139,2,99,99,3
14396,1053,3,PERSONNEL,ACORN,-1,140,2,99,99,3
14397,1050,1283,PERISCOPE,SHEPHERD,-1,141,2,99,99,3
14398,230,1405,CANDY,SUMMIT,-1,142,2,99,99,3


In [12]:
df_g3_test.query("session==0 and list==2")

,test_itemno1,test_itemno2,test_item1,correct_ans,order,pair_idx,test,list,session,subject,group
240,1507,-1,TRIGGER,0,-1,-1,1,2,0,0,3
241,1263,-1,SEASHORE,0,-1,-1,1,2,0,0,3
242,1062,-1,PIE,0,-1,-1,1,2,0,0,3
243,332,-1,COAL,0,-1,-1,1,2,0,0,3
244,1424,-1,SYRINGE,1,1,133,1,2,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...
355,1095,-1,POET,690,1,126,2,2,0,0,3
356,485,-1,DONOR,266,0,123,2,2,0,0,3
357,750,-1,JAR,452,1,107,2,2,0,0,3
358,1134,-1,PRUNE,1386,1,131,2,2,0,0,3


In [13]:
df_study = pd.concat([df_g1_study,df_g2_study,df_g3_study])
df_test = pd.concat([df_g1_test,df_g2_test,df_g3_test])

In [14]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,order,pair_idx,list,session,subject,group
0,1183,577,RELISH,FLEA,1,0,0,0,0,1
1,1531,41,TYPEWRITER,ARTERY,1,1,0,0,0,1
2,926,1213,MOUTH,ROUGE,1,2,0,0,0,1
3,1148,665,QUARTER,GROOM,1,3,0,0,0,1
4,56,774,ATTIC,KEYBOARD,1,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
14395,1111,294,PORTRAIT,CHESS,-1,139,2,99,99,3
14396,1053,3,PERSONNEL,ACORN,-1,140,2,99,99,3
14397,1050,1283,PERISCOPE,SHEPHERD,-1,141,2,99,99,3
14398,230,1405,CANDY,SUMMIT,-1,142,2,99,99,3


In [15]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,list,session,subject,group
0,1213,926,ROUGE,MOUTH,0,1,-1,1,0,0,0,1
1,830,717,LIST,HORIZON,1,1,34,1,0,0,0,1
2,577,1183,FLEA,RELISH,1,1,0,1,0,0,0,1
3,588,439,FOLDER,DANDRUFF,1,0,25,1,0,0,0,1
4,687,1311,HANGER,SIRLOIN,1,1,22,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
35995,75,-1,BALLOON,NaN,1284,0,123,2,2,99,99,3
35996,470,-1,DIPLOMA,NaN,224,0,119,2,2,99,99,3
35997,143,-1,BOARD,NaN,1526,1,126,2,2,99,99,3
35998,478,-1,DOCTOR,NaN,350,0,102,2,2,99,99,3


In [16]:
# save
with open('../Data/simuS1_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)